In [1]:
import sys
from os import listdir
import os

from xml.dom.minidom import parse
import pandas as pd

In [2]:
#datadir = "data/Train/MedLine"
datadir = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Train/DrugBank/"
print(datadir)

/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Train/DrugBank/


# Load data

In [3]:
trees = []
for f in listdir(datadir):
    # parse XML file
    tree = parse(datadir + "/" + f)
    trees.append(tree)

In [4]:
items = []
for tree in trees:
    for s in tree.getElementsByTagName("sentence"):
        s_id = s.attributes["id"].value
        s_txt = s.attributes["text"].value
        for e in s.getElementsByTagName("entity"):
            e_id = e.attributes["id"].value
            e_offset = e.attributes["charOffset"].value 
            e_name = e.attributes["text"].value
            e_type = e.attributes["type"].value
            item = {"s_id": s_id, "s_txt": s_txt, "e_id": e_id, "e_offset": e_offset, "e_name": e_name, "e_type": e_type}
            items.append(item)

In [5]:
df = pd.DataFrame(items)
df.shape

(12929, 6)

## Entities with sentences

In [6]:
df.head(10)

,e_id,e_name,e_offset,e_type,s_id,s_txt
0,DDI-DrugBank.d481.s0.e0,calcium,25-31,drug,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or..."
1,DDI-DrugBank.d481.s0.e1,EMCYT,82-86,brand,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or..."
2,DDI-DrugBank.d419.s0.e0,allopurinol,33-43,drug,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...
3,DDI-DrugBank.d419.s0.e1,ampicillin,49-58,drug,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...
4,DDI-DrugBank.d419.s0.e2,ampicillin,175-184,drug,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...
5,DDI-DrugBank.d419.s1.e0,ampicillin,45-54,drug,DDI-DrugBank.d419.s1,It is not known whether this potentiation of a...
6,DDI-DrugBank.d419.s1.e1,allopurinol,73-83,drug,DDI-DrugBank.d419.s1,It is not known whether this potentiation of a...
7,DDI-DrugBank.d419.s2.e0,AUGMENTIN XR,33-44,brand,DDI-DrugBank.d419.s2,"In controlled clinical trials of AUGMENTIN XR,..."
8,DDI-DrugBank.d419.s2.e1,allopurinol,80-90,drug,DDI-DrugBank.d419.s2,"In controlled clinical trials of AUGMENTIN XR,..."
9,DDI-DrugBank.d419.s2.e2,AUGMENTIN XR,96-107,brand,DDI-DrugBank.d419.s2,"In controlled clinical trials of AUGMENTIN XR,..."


In [7]:
df["e_type"].value_counts()

drug      8197
group     3206
brand     1423
drug_n     103
Name: e_type, dtype: int64

In [8]:
df["e_name"].value_counts()[:10]

digoxin          176
warfarin         176
phenytoin        157
lithium          119
theophylline     115
ketoconazole     107
cimetidine        89
carbamazepine     81
erythromycin      79
aspirin           79
Name: e_name, dtype: int64

In [9]:
df.describe()

,e_id,e_name,e_offset,e_type,s_id,s_txt
count,12929,12929,12929,12929,12929,12929
unique,12929,2655,3957,4,4683,4505
top,DDI-DrugBank.d415.s15.e0,digoxin,0-9,drug,DDI-DrugBank.d64.s87,Drugs that reportedly may increase oral antico...
freq,1,176,267,8197,55,55


## Entities and type 

In [10]:
df_e = df[["e_name", "e_type"]].drop_duplicates()
df_e.head(10)

,e_name,e_type
0,calcium,drug
1,EMCYT,brand
2,allopurinol,drug
3,ampicillin,drug
7,AUGMENTIN XR,brand
12,broad-spectrum antibiotics,group
14,contraceptives,group
15,PROCRIT,brand
16,central nervous system depressants,group
17,benzodiazepines,group


In [11]:
# there are names that have more than one type
df_e.describe()

,e_name,e_type
count,2719,2719
unique,2655,4
top,iodine,drug
freq,2,1393


In [12]:
df[df["e_name"]=="neuroleptic"].head()

,e_id,e_name,e_offset,e_type,s_id,s_txt
6272,DDI-DrugBank.d224.s6.e1,neuroleptic,96-106,group,DDI-DrugBank.d224.s6,An additive hypotensive effect has been report...
8689,DDI-DrugBank.d99.s19.e2,neuroleptic,106-116,drug,DDI-DrugBank.d99.s19,Hyperpyrexia has been reported when amitriptyl...


## Feature analysis

### Upper case

In [13]:
df_e['is_upper'] = df_e['e_name'].apply(lambda x : x.isupper())
df_e.head()

,e_name,e_type,is_upper
0,calcium,drug,False
1,EMCYT,brand,True
2,allopurinol,drug,False
3,ampicillin,drug,False
7,AUGMENTIN XR,brand,True


In [14]:
df_e.groupby(['e_type', 'is_upper']).count()

e_name
e_type is_upper        
brand  False        225
       True         165
drug   False       1356
       True          37
drug_n False         57
       True           9
group  False        852
       True          18

### Suffixes

In [15]:
df_e['suffix_5'] = df_e['e_name'].apply(lambda x : x[-5:].lower())
df_e.head()

,e_name,e_type,is_upper,suffix_5,suffix_4
0,calcium,drug,False,lcium,cium
1,EMCYT,brand,True,emcyt,mcyt
2,allopurinol,drug,False,rinol,inol
3,ampicillin,drug,False,illin,llin
7,AUGMENTIN XR,brand,True,in xr,n xr


In [16]:
def getColumAsList(e_type, column):
    tmp = df_e[df_e['e_type'] == e_type]
    return tmp[column].tolist()

def intersection(x1, x2):
    return list(set(x1).intersection(x2))

def disjunction(x, y):
    return [item for item in x if item not in y]

In [17]:
def getGroupList(column):
    res = {}
    for g in ["brand", "drug", "drug_n", "group"]:
        res[g] = getColumAsList(g, column)
    return res

type_suffix_5 = getGroupList("suffix_5")
for k in type_suffix_5.keys():
    s = list(set(type_suffix_5[k])) #uniques
    type_suffix_5[k] = s
    print(k, len(type_suffix_5[k]))

brand 360
drug 498
drug_n 47
group 193


In [18]:
import itertools
combs = list(itertools.combinations(type_suffix_5.keys(), 2))
print(combs)

[('brand', 'drug'), ('brand', 'drug_n'), ('brand', 'group'), ('drug', 'drug_n'), ('drug', 'group'), ('drug_n', 'group')]


In [19]:
for (c1, c2) in combs:
    x1 = type_suffix_5[c1]
    x2 = type_suffix_5[c2]
    inter = intersection(x1, x2)
    print("Intersection [{}-{}] = {}, {}, {}".format(c1,c2,len(inter),len(x1),len(x2)))

Intersection [brand-drug] = 33, 360, 498
Intersection [brand-drug_n] = 3, 360, 47
Intersection [brand-group] = 14, 360, 193
Intersection [drug-drug_n] = 31, 498, 47
Intersection [drug-group] = 62, 498, 193
Intersection [drug_n-group] = 3, 47, 193


In [20]:
new_type_suffix_5 = {}
all_keys = type_suffix_5.keys()
for k in type_suffix_5:
    x = type_suffix_5[k]
    y = []
    others = [x for x in all_keys if x != k]
    for o in others:
        y = y + type_suffix_5[o]
    new_type_suffix_5[k] = disjunction(x, y)
    
    new_length = len(new_type_suffix_5[k])
    old_length = len(type_suffix_5[k])
    print("Suffix_5 {} = {}, {}".format(k, new_length, (new_length / old_length)))

Suffix_5 brand = 322, 0.8944444444444445
Suffix_5 drug = 385, 0.7730923694779116
Suffix_5 drug_n = 16, 0.3404255319148936
Suffix_5 group = 126, 0.6528497409326425


In [21]:
initial_drug_suffixes = ["idase", "idone", "uride", "ogens", "rinol", "amate", "lones", "pamil", "olone", "parin", "ssant", "udine",
            "D", "etron", "adiol", "feine", "pines", "zines", "toxin", "MAO", "opram", "ophen", "sides", "talis",
            "ulant", "ylate", "osine", "oxide", "caine", "illin", "itant", "limus", "pride", "sulin", "oride", "abine",
            "hrine", "iazem", "atory", "oidal", "emide", "hanol", "phine", "SAIDs", "coxib", "necid", "nists", "esium",
            "acids", "nolol", "nafil", "azine", "exate", "rates", "cking", "lcium", "azide", "zepam", "arone", "rofen",
            "ampin", "ergic", "roids", "tamin", "adine", "odium", "bital", "pirin", "lants", "orine", "mines", "zolam",
            "apine", "cohol", "ckers", "ipine", "acid", "yclic", "otics", "tives", "xacin", "etine", "epine", "drugs",
            "tatin", "thium", "lline", "amide", "sants", "etics", "itors", "ytoin", "gents", "navir", "goxin", "farin",
            "mycin", "idine", "amine", "azole"]
print(len(initial_drug_suffixes), len(new_type_suffix_5["drug"]))

100 385


In [22]:
len(intersection(initial_drug_suffixes, new_type_suffix_5["drug"]))

38

In [23]:
#for k in new_type_suffix_5:
#    with open(k+'_suffix_5_DrugBank.txt', 'w') as f:
#            for item in new_type_suffix_5[k]:
#                f.write("%s\n" % item)

In [24]:
loaded = {}
for k in new_type_suffix_5:
    with open(k+'_suffix_5_DrugBank.txt', 'r') as f:
        loaded[k] = f.read().splitlines()

In [25]:
df_e[df_e["e_name"].str.contains("propranolo")]

,e_name,e_type,is_upper,suffix_5,suffix_4
1045,propranolol,drug,False,nolol,olol


### Plural

In [32]:
df_e['is_plural'] = df_e['e_name'].apply(lambda x : x[-1:] == 's')
df_e.head(10)

,e_name,e_type,is_upper,suffix_5,suffix_4,is_plural
0,calcium,drug,False,lcium,cium,False
1,EMCYT,brand,True,emcyt,mcyt,False
2,allopurinol,drug,False,rinol,inol,False
3,ampicillin,drug,False,illin,llin,False
7,AUGMENTIN XR,brand,True,in xr,n xr,False
12,broad-spectrum antibiotics,group,False,otics,tics,True
14,contraceptives,group,False,tives,ives,True
15,PROCRIT,brand,True,ocrit,crit,False
16,central nervous system depressants,group,False,sants,ants,True
17,benzodiazepines,group,False,pines,ines,True


In [33]:
df_e.groupby(["e_type", "is_plural"]).count()

e_name  is_upper  suffix_5  suffix_4
e_type is_plural                                      
brand  False         383       383       383       383
       True            7         7         7         7
drug   False        1355      1355      1355      1355
       True           38        38        38        38
drug_n False          64        64        64        64
       True            2         2         2         2
group  False         227       227       227       227
       True          643       643       643       643